<a href="https://colab.research.google.com/github/nauvalZulfikar/Algotrading-projects/blob/main/Mean_reversion_strategy_using_Bollinger_Bands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Download historical data
def get_data(symbol, start, end):
    df = yf.download(symbol, start=start, end=end)
    df.columns = [df.columns[i][0] for i in range(len(df.columns))]
    df['Returns'] = df['Close'].pct_change()
    return df

# Compute Bollinger Bands
def calculate_bollinger_bands(df, window=20, num_std=2):
    df['SMA'] = df['Close'].rolling(window).mean()
    df['Upper Band'] = df['SMA'] + (df['Close'].rolling(window).std() * num_std)
    df['Lower Band'] = df['SMA'] - (df['Close'].rolling(window).std() * num_std)
    return df

# Compute RSI
def calculate_rsi(df, period=14):
    delta = df['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Implement Mean Reversion Strategy
def mean_reversion_strategy(df):
    df['Position'] = 0
    df.loc[(df['Close'] < df['Lower Band']) & (df['RSI'] < 30), 'Position'] = 1  # Buy Signal
    df.loc[(df['Close'] > df['Upper Band']) & (df['RSI'] > 70), 'Position'] = -1  # Sell Signal
    df['Strategy Returns'] = df['Position'].shift(1) * df['Returns']
    return df

# Backtest Performance
def backtest(df, initial_capital=10000):
    df['Cumulative Returns'] = (1 + df['Strategy Returns']).cumprod() * initial_capital
    final_return = df['Cumulative Returns'].iloc[-1]

    # Calculate backtesting metrics
    df['Cumulative Max'] = df['Cumulative Returns'].cummax()
    df['Drawdown'] = (df['Cumulative Returns'] / df['Cumulative Max']) - 1
    total_return = df['Cumulative Returns'].iloc[-1] - initial_capital
    sharpe_ratio = df['Strategy Returns'].mean() / df['Strategy Returns'].std() * np.sqrt(252)
    max_drawdown = df['Drawdown'].min()
    win_rate = (df['Strategy Returns'] > 0).sum() / df['Strategy Returns'].count()

    print(f'Final Portfolio Value: ${final_return:.2f}')
    print(f'Total Return: ${total_return:.2f}')
    print(f'Sharpe Ratio: {sharpe_ratio:.2f}')
    print(f'Max Drawdown: {max_drawdown:.2%}')
    print(f'Win Rate: {win_rate:.2%}')

    return df, final_return

# Plot results
def plot_results(df, symbol):
    fig, ax1 = plt.subplots(figsize=(12,6))

    # Plot price and Bollinger Bands
    ax1.plot(df.index, df['Close'], label=f'{symbol} Price', color='black')
    ax1.plot(df.index, df['Upper Band'], label='Upper Band', linestyle='dashed', color='red')
    ax1.plot(df.index, df['Lower Band'], label='Lower Band', linestyle='dashed', color='green')
    ax1.fill_between(df.index, df['Lower Band'], df['Upper Band'], color='gray', alpha=0.2)
    ax1.scatter(df.index[df['Position'] == 1], df['Close'][df['Position'] == 1], label='Buy Signal', marker='^', color='green', s=100, alpha=1)
    ax1.scatter(df.index[df['Position'] == -1], df['Close'][df['Position'] == -1], label='Sell Signal', marker='v', color='red', s=100, alpha=1)
    ax1.set_ylabel('Price')
    ax1.legend(loc='upper left')

    # Create secondary axis for drawdown
    ax2 = ax1.twinx()
    ax2.plot(df.index, df['Drawdown'], label='Drawdown', color='blue', alpha=0.5)
    ax2.fill_between(df.index, df['Drawdown'], 0, color='blue', alpha=0.2)
    ax2.set_ylabel('Drawdown')
    ax2.legend(loc='lower right')

    plt.title(f'Mean Reversion Strategy on {symbol} with Drawdown')
    plt.show()

# Run the strategy
symbol = 'AAPL'  # Change symbol if needed
start = '2020-01-01'
end = '2025-01-01'

df = get_data(symbol, start, end)
df = calculate_bollinger_bands(df)
df = calculate_rsi(df)
df = mean_reversion_strategy(df)
df, final_value = backtest(df)

plot_results(df, symbol)